In [58]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import glob
from tqdm import tnrange, tqdm_notebook
import pysam

In [59]:
jxc_dir = '/home/elvannostrand/data/ENCODE/RNAseq/scripts/exon_junction_counts/encode_rnaseq_psis_20170321/'
jxc_i = glob.glob(os.path.join(jxc_dir,'*.bam.namesort.bam.int_jxn_counts'))
jxc = glob.glob(os.path.join(jxc_dir,'*.bam.namesort.bam.jxn_counts'))
bam = '/home/bay001/projects/encode/analysis/tests/eric_retained_intron/ENCFF385UCY.sorted.bam'

In [60]:
all_controls = pd.read_table('/projects/ps-yeolab3/encode/k562_brenton-graveley_ambiguous_bams_for_integrated_analysis.txt')
all_controls_r1 = set(all_controls['control_rep1'])
all_controls_r2 = set(all_controls['control_rep2'])
ac = all_controls_r1.union(all_controls_r2)
ac = set(ac)

In [68]:
stop_at = 10
bad = []
ctrl_bams = ac

c = 0
ac = ['ENCFF157TKC.bam']
progress = tnrange(len(ac))
for b in ac:
    original = '/projects/ps-yeolab3/encode/rnaseq/shrna_knockdown_graveley_tophat/{}'.format(b)
    bam = '/home/bay001/projects/encode/analysis/tests/eric_retained_intron/{}'.format(b)
    if not os.path.exists(bam):
        ! ln -s $original $bam
    if not os.path.exists(bam + '.bai'):
        ! samtools index $bam
    i = '/home/elvannostrand/data/ENCODE/RNAseq/scripts/exon_junction_counts/encode_rnaseq_psis_20170321/{}.namesort.bam.int_jxn_counts'.format(b)
    samfile = pysam.AlignmentFile(bam, "rb", )
    # for jct in dfi['jct'].head():
    dfi = pd.read_table(i, names = ['chrom','strand','jct','num'])
    # dfi = pd.DataFrame(dfi.ix[3]).T
    for col, row in dfi.iterrows():
        c += 1
        names = []
        for read in samfile.fetch(row['chrom'], row['jct'], row['jct']+1):
            # print(read.get_reference_positions()[0], row['jct'])
            if 'N' not in read.cigarstring:
                if row['jct'] - read.get_reference_positions()[0] > 10 and read.get_reference_positions()[-1] - row['jct'] > 10: # sufficient overhang near read start
                    names.append(read.query_name)
        if c > stop_at:
            break
        if (len(set(names)) != row['num']):
            bad.append([b,set(names), row['num'], row['jct']])
        
        
    if c > stop_at:
        break
    # print("{}:{} ".format(b,len(bad))),
    
    progress.update(1)

In [69]:
bad

[['ENCFF157TKC.bam',
  {'HWI-D00304:123:C8LH5ANXX:6:1108:12589:77972',
   'HWI-D00304:123:C8LH5ANXX:6:1111:14318:66337',
   'HWI-D00304:123:C8LH5ANXX:6:2302:2548:16505'},
  2,
  40875508],
 ['ENCFF157TKC.bam',
  {'HWI-D00304:123:C8LH5ANXX:6:1301:5808:59002',
   'HWI-D00304:123:C8LH5ANXX:6:2105:3131:8118'},
  1,
  9784332],
 ['ENCFF157TKC.bam',
  {'HWI-D00304:123:C8LH5ANXX:6:1116:18534:16753',
   'HWI-D00304:123:C8LH5ANXX:6:1204:10267:54805',
   'HWI-D00304:123:C8LH5ANXX:6:1214:14264:68408',
   'HWI-D00304:123:C8LH5ANXX:6:2210:8746:9622',
   'HWI-D00304:123:C8LH5ANXX:6:2307:19433:25314',
   'HWI-D00304:123:C8LH5ANXX:6:2313:6061:9321',
   'HWI-D00304:123:C8LH5ANXX:6:2316:6682:36497'},
  2,
  53678934],
 ['ENCFF157TKC.bam',
  {'HWI-D00304:123:C8LH5ANXX:6:1312:8410:95013',
   'HWI-D00304:123:C8LH5ANXX:6:2109:2126:45635',
   'HWI-D00304:123:C8LH5ANXX:6:2110:18064:4818'},
  2,
  143348887],
 ['ENCFF157TKC.bam',
  {'HWI-D00304:123:C8LH5ANXX:6:1107:1303:24149',
   'HWI-D00304:123:C8LH5ANXX:6:2

In [6]:
def make_bedtool(eric, offset=0):
    chrom, strand, pos = eric.split(':')
    start, end = pos.split('-')
    return pybedtools.create_interval_from_list([
            chrom, str(int(start) + offset), str(int(start) + offset + 1), 'start', '0', strand
        ])
def count_jcts(jct_df):
    bts = []
    for junc in jct_df['jct']:
        bts.append(make_bedtool(junc))
    return pybedtools.BedTool(bts)

In [ ]:
bts = count_jcts(df).sort()

In [ ]:
b = pybedtools.BedTool(bam)
bts[0]

In [ ]:
btx = bts.intersect(b, sorted=True, wa=True, wb=True)

In [ ]:
dfx = btx.to_dataframe()

In [ ]:
single_junction_site = pybedtools.create_interval_from_list(['chr1','40875508','40875509','name','0','+'])
single_junction_site = pybedtools.BedTool(single_junction_site)
single_junction_site

In [ ]:
dfy = single_junction_site.intersect(b, wa=True, wb=True)
dfy

In [ ]:

samfile = pysam.AlignmentFile(bam, "rb", )
c = 0
for read in samfile.fetch('chr1', 40875507, 40875508):
     if 'N' not in read.cigarstring:
        # print(read.get_reference_positions()[0] - 40875508, read.query_name, read.flag, read.cigarstring, read.get_reference_positions()[0])
        c+=1
        # print(read.query_name)
print(c)

In [ ]:
i = '/home/elvannostrand/data/ENCODE/RNAseq/scripts/exon_junction_counts/encode_rnaseq_psis_20170321/ENCFF475GCB.bam.namesort.bam.int_jxn_counts'

dfi = pd.read_table(i, names = ['chrom','strand','jct','num'])
dfi.head()

[['ENCFF157TKC.bam', set(['HWI-D00304:123:C8LH5ANXX:6:2210:8746:9622', 'HWI-D00304:123:C8LH5ANXX:6:1116:18534:16753', 'HWI-D00304:123:C8LH5ANXX:6:2316:6682:36497', 'HWI-D00304:123:C8LH5ANXX:6:2313:6061:9321', 'HWI-D00304:123:C8LH5ANXX:6:1204:10267:54805', 'HWI-D00304:123:C8LH5ANXX:6:2307:19433:25314', 'HWI-D00304:123:C8LH5ANXX:6:1214:14264:68408'])]]


In [56]:
set(names)

{'HWI-D00304:123:C8LH5ANXX:6:1116:18534:16753',
 'HWI-D00304:123:C8LH5ANXX:6:1204:10267:54805',
 'HWI-D00304:123:C8LH5ANXX:6:1214:14264:68408',
 'HWI-D00304:123:C8LH5ANXX:6:2210:8746:9622',
 'HWI-D00304:123:C8LH5ANXX:6:2307:19433:25314',
 'HWI-D00304:123:C8LH5ANXX:6:2313:6061:9321',
 'HWI-D00304:123:C8LH5ANXX:6:2316:6682:36497'}

In [51]:
pd.DataFrame(dfi)

,chrom,strand,jct,num
3,chr1,+,53678934,2


In [52]:
i

'/home/elvannostrand/data/ENCODE/RNAseq/scripts/exon_junction_counts/encode_rnaseq_psis_20170321/ENCFF157TKC.bam.namesort.bam.int_jxn_counts'